## RAGAS for the Korean dataset

In [1]:
from ragas.metrics import (
    faithfulness,
    answer_correctness,
    answer_similarity
)
from langchain.chat_models import ChatOpenAI
from ragas import adapt
import os

os.environ["OPENAI_API_KEY"] = "YOUR OWN KEY" 

# llm used for adaptation
openai_model = ChatOpenAI(model_name="gpt-4")

adapt(metrics=[faithfulness, answer_correctness], language="korean", llm=openai_model)

In [2]:
print(faithfulness.nli_statements_message.to_string())

Your task is to judge the faithfulness of a series of statements based on a given context. For each statement you must return verdict as 1 if the statement can be directly inferred based on the context or 0 if the statement can not be directly inferred based on the context.

The output should be a well-formatted JSON instance that conforms to the JSON schema below.

As an example, for the schema {{"properties": {{"foo": {{"title": "Foo", "description": "a list of strings", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}
the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of the schema. The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Here is the output JSON schema:
```
{{"type": "array", "items": {{"$ref": "#/definitions/StatementFaithfulnessAnswer"}}, "definitions": {{"StatementFaithfulnessAnswer": {{"title": "StatementFaithfulnessAnswer", "type": "object", "properties": {{"statement": {{"title": "Statement", "descr

## Evaluation sample

In [3]:
from datasets import load_dataset, Dataset

data_samples = {
    "question": [
        "조석이 기르던 고양이 이름은 뭐야?",
        "조석은 몇년도에 고양이를 키웠어?",
        "조석은 몇년도에 고양이를 키웠어?",
        "조석은 몇년도에 고양이를 키웠어?",
    ],
    "answer": ["게레로", "2005년", "2007년", "2007"],
    "contexts": [
        [
            "게레로는 2007년 조석이 잠시 맡아 키우던 고양이. 고향 부모 집에 맡겼고 오래잖아 가출하여 4년 째 종적이 묘연해진 상태다. 지금의 센세이션과 같은 캐릭터를 갖고 있었다고 보면 된다. 이에 대해서는 그동안 단순 고양이 가출 정도로만 취급되다 2011년 7월 말에 전말이 알려졌다. 자세한 것은 위의 사과문 게재 단락과 조석(만화가)조석 문서 참고."
        ],
        [
            "게레로는 2007년 조석이 잠시 맡아 키우던 고양이. 고향 부모 집에 맡겼고 오래잖아 가출하여 4년 째 종적이 묘연해진 상태다. 지금의 센세이션과 같은 캐릭터를 갖고 있었다고 보면 된다. 이에 대해서는 그동안 단순 고양이 가출 정도로만 취급되다 2011년 7월 말에 전말이 알려졌다. 자세한 것은 위의 사과문 게재 단락과 조석(만화가)조석 문서 참고."
        ],
        [
            " 게레로는 2007년 조석이 잠시 맡아 키우던 고양이. 고향 부모 집에 맡겼고 오래잖아 가출하여 4년 째 종적이 묘연해진 상태다. 지금의 센세이션과 같은 캐릭터를 갖고 있었다고 보면 된다. 이에 대해서는 그동안 단순 고양이 가출 정도로만 취급되다 2011년 7월 말에 전말이 알려졌다. 자세한 것은 위의 사과문 게재 단락과 조석(만화가)조석 문서 참고."
        ],
        [
            "게레로는 2007년 조석이 잠시 맡아 키우던 고양이. 고향 부모 집에 맡겼고 오래잖아 가출하여 4년 째 종적이 묘연해진 상태다. 지금의 센세이션과 같은 캐릭터를 갖고 있었다고 보면 된다. 이에 대해서는 그동안 단순 고양이 가출 정도로만 취급되다 2011년 7월 말에 전말이 알려졌다. 자세한 것은 위의 사과문 게재 단락과 조석(만화가)조석 문서 참고."
        ],
    ],
    "ground_truth": ["게레로", "2007년", "2007년", "2007년"],
}

dataset = Dataset.from_dict(data_samples)

In [6]:
from ragas import evaluate

ragas_score = evaluate(dataset, metrics=[faithfulness, answer_correctness, answer_similarity])

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
Failed to parse output. Returning None.
Failed to parse output. Returning None.


In [7]:
ragas_score.to_pandas()

,question,answer,contexts,ground_truth,faithfulness,answer_correctness,answer_similarity
0,조석이 기르던 고양이 이름은 뭐야?,게레로,[게레로는 2007년 조석이 잠시 맡아 키우던 고양이. 고향 부모 집에 맡겼고 오래...,게레로,0.0,1.000000,1.000000
1,조석은 몇년도에 고양이를 키웠어?,2005년,[게레로는 2007년 조석이 잠시 맡아 키우던 고양이. 고향 부모 집에 맡겼고 오래...,2007년,NaN,0.979297,0.917188
2,조석은 몇년도에 고양이를 키웠어?,2007년,[ 게레로는 2007년 조석이 잠시 맡아 키우던 고양이. 고향 부모 집에 맡겼고 오...,2007년,NaN,1.000000,1.000000
3,조석은 몇년도에 고양이를 키웠어?,2007,[게레로는 2007년 조석이 잠시 맡아 키우던 고양이. 고향 부모 집에 맡겼고 오래...,2007년,NaN,0.978104,0.912417


## Test dataset synthesis

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("namuwiki.txt")
documents = loader.load()

In [4]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# documents = load your documents

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(generator_llm, critic_llm, embeddings)

In [10]:
testset_pandas = testset.to_pandas()
testset_pandas.to_csv("testset_generated_0611.csv")

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,Who are the relatives of 조석 in 마음의소리?,[개요: 마음의소리의 등장인물을 정리한 문서.\n조석의 가족: 조석의 가족 및 친척...,"조석의 가족 및 친척들. 현재 시점에서는 조석, 형 조준, 부모님, 아내 애봉이, ...",simple,[{'source': 'namuwiki.txt'}],True
1,"What is the role of ""소대장"" in the comic, given ...","[은 그냥 옛날부터 엑스트라로 등장하던 이름인 ""민철""을 거꾸로 한 것으로 보인다....",The role of '소대장' in the comic is to occasiona...,multi_context,[{'source': 'namuwiki.txt'}],True
